In [2]:
import numpy as np
import mnist_reader
X_train, y_train = mnist_reader.load_mnist('data/fashion', kind='train')
X_test, y_test = mnist_reader.load_mnist('data/fashion', kind='t10k')

In [14]:
print(y_train[1])
print(X_train[1])

0


In [16]:
def feed_foward(thetas, X):
    a = [X]
    for i in range(len(thetas)):
        a.append(
            sigmoid(
                np.matmul(
                    np.hstack((
                        np.ones(len(X)).reshape(len(X), 1),
                    )),
                    thetas[i].T
                )
            )
        )
        return a

In [ ]:
def inflate_matrixes(flat_thetas, shapes):
    layers = len(shapes) + 1
    sizes = [shapes[0] + shape[1] for shape in shapes]
    steps = np.zeros(layers, dtype=int)
    
    for i in range(layers - 1):
        steps[i + 1] = steps[i] + sizes[i]
    return [
        flat_thetas[steps[i]: steps[i + 1]].reshape(*shapes[i])
        for i in range(layers - 1)
    ]

In [17]:
def cost_function(flat_thetas, shapes, X, Y):
    a = feed_foward(
        inflate_matrixes(flat_thetas, shapes),
        X
    )
    return -(Y * np.log(a[-1]) + (1 - Y) * np.log(1 - a[-1])).sum() / len(X)

In [ ]:
def cost_function_jacobian(flat_thetas, shapes, X, Y):
    m, layers = len(X), len(shapes) + 1
    thetas = inflate_matrixes(flat_thetas, shapes)
    a = feed_foward(thetas, X)
    deltas = [*range(layers - 1), a[-1] - Y]
    #2.4
    for i in range(layers - 2, 0, -1):
        a
    #2.5 y 3.0
    return flatten_list_of_arrays([])

In [ ]:
float_thetas = nn.flatten_list_of arrays([
    np.random.rand(*theta_shape)
    for theta_shape in theta_shapes
])

result = op.minimize(
    fun=nn.cost_function,
    x0=flat_thetas,
    args=(theta_shapes,X,Y),
    method='L-BFGS-B',
    jac=nn.cost_function_jacobian,
    options={'disp':True, 'maxiter': 3000}
)